# Information Retrieval Evaluation Pipeline

This notebook implements a comprehensive evaluation pipeline for information retrieval systems using FAISS indices and BGE-M3 embeddings. The pipeline includes:
- Retrieval evaluation with multiple metrics
- Visualization of results
- Detailed performance analysis by categories
- Report generation

First, let's set up our environment and install required dependencies.

In [9]:
!pip install torch transformers sentence-transformers rank_bm25 nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [10]:
!pip install faiss-cpu FlagEmbedding tqdm numpy matplotlib seaborn

# from google.colab import drive
# drive.mount('/content/drive')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Import Required Libraries

Let's import all the necessary Python libraries and configure the environment.

In [1]:

import multiprocessing
import faiss
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os
from FlagEmbedding import BGEM3FlagModel
from app.utils.retrievers.evaluator import RetrievalEvaluator
from app.utils.retrievers.multi_step import QueryRewriterRetrieverSubcategory, FilteredLaw2StepRetriever, FilteredPart2StepRetriever,\
RankLaw2StepRetriever, RankPart2StepRetriever
from app.utils.retrievers.retriever import Retriever
from langchain_openai import ChatOpenAI

/home/naif/miniconda3/envs/legal-assistant/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
faiss.omp_set_num_threads(12)

## Define Data Classes and Helper Functions

First, let's define our evaluation metrics dataclass and implement the core evaluation class.

## Visualization Functions

Let's implement functions to visualize our evaluation results.

## Run Evaluation Pipeline

Now let's run the evaluation pipeline. First, we need to:
1. Initialize the BGE-M3 model
2. Set up paths for the FAISS index and QA dataset
3. Create an evaluator instance
4. Run the evaluation
5. Generate visualizations and reports

In [3]:
# Configurations
retriever_model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)
metric_type = 'ip'
metric_suffix = 'V2' if metric_type == 'l2' else ''

index_path = 'data/m3_legal_faiss.index'
qa_dataset_path ='data/evaluation_data/relational_qa_dataset_v2.json' #
documents_path = 'data/saudi_laws_scraped.json'
k_values=[1, 3, 5, 10, 20, 40, 80, 100, 200]
# Load QA dataset
with open(qa_dataset_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
    # We assume qa_pairs is a list of dicts, e.g.:
    # [{"query": "...", "relevant_ids": [123, 456]}, ...]
    qa_pairs = data.get('qa_pairs', [])
print(f"✓ Loaded {len(qa_pairs)} QA pairs")

print(f"Using {metric_type.upper()} metric with index: {index_path}")
articles_index = faiss.read_index(index_path)
retriever = Retriever(
    faiss_index=articles_index,
    documents_path=documents_path,
    embeddings_model=retriever_model
)
# llm = ChatOpenAI(_(self
#         openai_api_base ="http://localhost:11434/v1",
#         api_key ="good",
#         model="hf.co/unsloth/Qwen3-30B-A3B-Instruct-2507-GGUF:Q4_K_M",
#         temperature=0,
#         max_tokens=1000
#     )
# qw_retriever = QueryRewriterRetrieverSubcategory(
#         llm=llm,
#         faiss_index_path=index_path,
#         documents_path=documents_path,
#         embeddings_model=retriever_model,
# )

# part_2_step_retriever = FilteredPart2StepRetriever(
#     parts_faiss_index=faiss.read_index('data/m3_legal_faiss_parts.index'),
#     articles_faiss_index=articles_index,
#     id_to_part_path='data/m3_corpus_id_to_part_id_parts.json',
#     documents_path=documents_path,
#     embeddings_model=retriever_model,
# )


# law_2_step_retriever = FilteredLaw2StepRetriever(
#     laws_faiss_index=faiss.read_index('data/m3_legal_faiss_laws.index'),
#     articles_faiss_index=articles_index,
#     documents_path=documents_path,
#     embeddings_model=retriever_model,
# )

part_2_step_retriever = RankPart2StepRetriever(
    parts_faiss_index=faiss.read_index('data/m3_legal_faiss_parts.index'),
    articles_faiss_index=articles_index,
    id_to_part_path='data/m3_corpus_id_to_part_id_parts.json',
    documents_path=documents_path,
    embeddings_model=retriever_model,
)

law_2_step_retriever = RankLaw2StepRetriever(
    laws_faiss_index=faiss.read_index('data/m3_legal_faiss_laws.index'),
    articles_faiss_index=articles_index,
    documents_path=documents_path,
    embeddings_model=retriever_model,
)

evaluator_dense = RetrievalEvaluator(
    name="dense",
    approach_desc="Dense with IP Metric",
    model_name="BAAI/bge-m3",
    retrieve_function =retriever.dense,
    qa_pairs=qa_pairs,
    k_values=k_values,
    is_baseline=True,
)


evaluator_hybrid = RetrievalEvaluator(
        name="hybrid_dense_sparse",
        approach_desc="Hybrid (Dense + Sparse)",
        model_name="bge-small + BM25",
        retrieve_function =retriever.hybrid,
        qa_pairs=qa_pairs,
        k_values=k_values,
)

# evaluator_qw_retriever = RetrievalEvaluator(
#         name="",
#         approach_desc="Query Rewriter+ filtering + Hybrid Retriever",
#         model_name="Qwen3-bge-small + BM25",
#         retrieve_function =qw_retriever.retrieve,
#         qa_pairs=qa_pairs,
#         k_values=k_values,
# )

evaluator_law_2_step = RetrievalEvaluator(
        name="law_2_step_retriever",
        approach_desc="Law 2-Step + Hybrid Retriever",
        model_name="BAAI/bge-m3 + BM25",
        retrieve_function =law_2_step_retriever.retrieve,
        qa_pairs=qa_pairs,
        k_values=k_values,
)

evaluator_part_2_step = RetrievalEvaluator(
        name="part_2_step_retriever",
        approach_desc="Part 2-Step + Hybrid Retriever",
        model_name="BAAI/bge-m3 + BM25",
        retrieve_function =part_2_step_retriever.retrieve,
        qa_pairs=qa_pairs,
        k_values=k_values,
)

Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 63103.87it/s]


✓ Loaded 280 QA pairs
Using IP metric with index: data/m3_legal_faiss.index
Initializing Retriever...
Loading documents and metadata...
✓ Loaded 16371 documents with metadata
✓ BM25 initialized successfully!
Initializing Retriever...
Loading documents and metadata...
✓ Loaded 16371 documents with metadata
✓ BM25 initialized successfully!
Initializing Retriever...
Loading documents and metadata...
✓ Loaded 16371 documents with metadata
✓ BM25 initialized successfully!


In [4]:
# reslts = evaluator_qw_retriever.evaluate_all()

In [5]:
# evaluator_qw_retriever.save_results('.')
# from app.utils.retrievers.evaluator import compare_approaches

# compare_approaches([evaluator_qw_retriever], metric_type="ex")

In [6]:
from app.utils.retrievers.evaluator import compare_approaches
approaches = [evaluator_dense, evaluator_hybrid, evaluator_law_2_step, evaluator_part_2_step]#, evaluator_law_2_step, evaluator_part_2_step]
EVAL_DIR = 'evaluation_pipeline/assets'
METRIC_TYPE = 'retrieval'
METRIC_SUFFIX = "relational_V0"
# 3. Run evaluations
print(f"\nStarting evaluation for {len(approaches)} approaches...")
for approach in approaches:
    approach.evaluate_all()


# 4. Save results (plot, report, details)
print("\nGenerating visualizations and saving results...")
for approach in approaches:
    approach.save_results(
        evaluation_dir=EVAL_DIR,
        metric_suffix=METRIC_SUFFIX,
        # METRIC_TYPE=METRIC_TYPE
    )

print("\n✓ Evaluation workflow completed successfully!")
# 5. Compare all approaches
compare_approaches(
    approaches,
    METRIC_TYPE
)


Starting evaluation for 4 approaches...
Evaluating 280 queries for 'dense'...


100%|██████████| 280/280 [00:07<00:00, 37.44it/s]


Evaluating 280 queries for 'hybrid_dense_sparse'...


100%|██████████| 280/280 [00:16<00:00, 17.42it/s]


Evaluating 280 queries for 'law_2_step_retriever'...


100%|██████████| 280/280 [00:22<00:00, 12.28it/s]


Evaluating 280 queries for 'part_2_step_retriever'...


100%|██████████| 280/280 [00:30<00:00,  9.31it/s]



Generating visualizations and saving results...
✓ Saved plot to evaluation_pipeline/assets/evaluation_metrics_dense_relational_V0.png
✓ Saved all results for 'dense' to evaluation_pipeline/assets
✓ Saved plot to evaluation_pipeline/assets/evaluation_metrics_hybrid_dense_sparse_relational_V0.png
✓ Saved all results for 'hybrid_dense_sparse' to evaluation_pipeline/assets
✓ Saved plot to evaluation_pipeline/assets/evaluation_metrics_law_2_step_retriever_relational_V0.png
✓ Saved all results for 'law_2_step_retriever' to evaluation_pipeline/assets
✓ Saved plot to evaluation_pipeline/assets/evaluation_metrics_part_2_step_retriever_relational_V0.png
✓ Saved all results for 'part_2_step_retriever' to evaluation_pipeline/assets

✓ Evaluation workflow completed successfully!

## METRIC COMPARISON: RETRIEVAL | BASELINE: Dense with IP Metric

| Metric        | Dense with IP Metric | Hybrid (Dense + Sparse) | Law 2-Step + Hybrid Retriever | Part 2-Step + Hybrid Retriever |
| -------------- | ----

| Metric        | Dense with IP Metric | Hybrid (Dense + Sparse) | Law 2-Step + Hybrid Retriever | Part 2-Step + Hybrid Retriever |
| -------------- | --------------------- | ------------------------ | ------------------------------ | ------------------------------- |
| Recall@1      | 0.0890               | 0.0915 (+2.8%)          | 0.0888 (-0.2%)                | 0.0922 (+3.6%)                 |
| Recall@3      | 0.1685               | 0.1750 (+3.9%)          | 0.1802 (+7.0%)                | 0.1732 (+2.8%)                 |
| Recall@5      | 0.2326               | 0.2397 (+3.1%)          | 0.2508 (+7.8%)                | 0.2380 (+2.3%)                 |
| Recall@10     | 0.3232               | 0.3405 (+5.3%)          | 0.3832 (+18.6%)               | 0.3466 (+7.2%)                 |
| Recall@20     | 0.4392               | 0.4625 (+5.3%)          | 0.5373 (+22.3%)               | 0.4635 (+5.5%)                 |
| Recall@40     | 0.5615               | 0.5838 (+4.0%)          | 0.6841 (+21.8%)               | 0.5828 (+3.8%)                 |
| Recall@80     | 0.6639               | 0.6813 (+2.6%)          | 0.7666 (+15.5%)               | 0.6834 (+2.9%)                 |
| Recall@100    | 0.6943               | 0.7081 (+2.0%)          | 0.7905 (+13.9%)               | 0.7093 (+2.2%)                 |
| Precision@1   | 0.2976               | 0.3164 (+6.3%)          | 0.3110 (+4.5%)                | 0.3190 (+7.2%)                 |
| Precision@3   | 0.2118               | 0.2181 (+3.0%)          | 0.2279 (+7.6%)                | 0.2163 (+2.1%)                 |
| Precision@5   | 0.1828               | 0.1893 (+3.5%)          | 0.1979 (+8.2%)                | 0.1882 (+2.9%)                 |
| Precision@10  | 0.1338               | 0.1418 (+6.0%)          | 0.1617 (+20.8%)               | 0.1432 (+7.0%)                 |
| Precision@20  | 0.1005               | 0.1056 (+5.1%)          | 0.1255 (+24.8%)               | 0.1058 (+5.2%)                 |
| Precision@40  | 0.0690               | 0.0719 (+4.3%)          | 0.0867 (+25.8%)               | 0.0718 (+4.2%)                 |
| Precision@80  | 0.0426               | 0.0435 (+2.3%)          | 0.0500 (+17.6%)               | 0.0436 (+2.5%)                 |
| Precision@100 | 0.0358               | 0.0365 (+1.9%)          | 0.0416 (+16.0%)               | 0.0366 (+2.1%)                 |
| NDCG@1        | 0.2976               | 0.3164 (+6.3%)          | 0.3110 (+4.5%)                | 0.3190 (+7.2%)                 |
| NDCG@3        | 0.2508               | 0.2591 (+3.3%)          | 0.2646 (+5.5%)                | 0.2575 (+2.7%)                 |
| NDCG@5        | 0.2583               | 0.2677 (+3.7%)          | 0.2740 (+6.1%)                | 0.2661 (+3.0%)                 |
| NDCG@10       | 0.2868               | 0.3004 (+4.7%)          | 0.3237 (+12.9%)               | 0.3019 (+5.3%)                 |
| NDCG@20       | 0.3342               | 0.3497 (+4.7%)          | 0.3879 (+16.1%)               | 0.3495 (+4.6%)                 |
| NDCG@40       | 0.3777               | 0.3933 (+4.1%)          | 0.4417 (+16.9%)               | 0.3924 (+3.9%)                 |
| NDCG@80       | 0.4093               | 0.4232 (+3.4%)          | 0.4674 (+14.2%)               | 0.4230 (+3.3%)                 |
| NDCG@100      | 0.4174               | 0.4306 (+3.2%)          | 0.4740 (+13.6%)               | 0.4302 (+3.1%)                 |
| Hit Rate@1    | 0.2976               | 0.3164 (+6.3%)          | 0.3110 (+4.5%)                | 0.3190 (+7.2%)                 |
| Hit Rate@3    | 0.4638               | 0.4879 (+5.2%)          | 0.5040 (+8.7%)                | 0.4906 (+5.8%)                 |
| Hit Rate@5    | 0.5791               | 0.5791 (+0.0%)          | 0.5871 (+1.4%)                | 0.5791 (+0.0%)                 |
| Hit Rate@10   | 0.6810               | 0.7078 (+3.9%)          | 0.7212 (+5.9%)                | 0.7051 (+3.5%)                 |
| Hit Rate@20   | 0.7828               | 0.8257 (+5.5%)          | 0.8365 (+6.8%)                | 0.8231 (+5.1%)                 |
| Hit Rate@40   | 0.8633               | 0.8660 (+0.3%)          | 0.8740 (+1.2%)                | 0.8660 (+0.3%)                 |
| Hit Rate@80   | 0.8901               | 0.9008 (+1.2%)          | 0.9142 (+2.7%)                | 0.9008 (+1.2%)                 |
| Hit Rate@100  | 0.9008               | 0.9088 (+0.9%)          | 0.9249 (+2.7%)                | 0.9088 (+0.9%)                 |
| MRR           | 0.4214               | 0.4382 (+4.0%)          | 0.4411 (+4.7%)                | 0.4381 (+3.9%)                 |
| MAP           | 0.2412               | 0.2525 (+4.7%)          | 0.2851 (+18.2%)               | 0.2515 (+4.3%)                 |

| Metric        | Dense with IP Metric | Hybrid (Dense + Sparse) | Law 2-Step + Hybrid Retriever | Part 2-Step + Hybrid Retriever |
| -------------- | --------------------- | ------------------------ | ------------------------------ | ------------------------------- |
| Recall@1      | 0.0890               | 0.0915 (+2.8%)          | 0.0869 (-2.4%)                | 0.0922 (+3.6%)                 |
| Recall@3      | 0.1685               | 0.1750 (+3.9%)          | 0.1786 (+6.0%)                | 0.1732 (+2.8%)                 |
| Recall@5      | 0.2326               | 0.2397 (+3.1%)          | 0.2492 (+7.1%)                | 0.2380 (+2.3%)                 |
| Recall@10     | 0.3232               | 0.3405 (+5.3%)          | 0.3899 (+20.6%)               | 0.3466 (+7.2%)                 |
| Recall@20     | 0.4392               | 0.4625 (+5.3%)          | 0.5503 (+25.3%)               | 0.4635 (+5.5%)                 |
| Recall@40     | 0.5615               | 0.5838 (+4.0%)          | 0.6788 (+20.9%)               | 0.5828 (+3.8%)                 |
| Recall@80     | 0.6639               | 0.6813 (+2.6%)          | 0.7622 (+14.8%)               | 0.6834 (+2.9%)                 |
| Recall@100    | 0.6943               | 0.7081 (+2.0%)          | 0.7807 (+12.4%)               | 0.7093 (+2.2%)                 |
| Precision@1   | 0.2976               | 0.3164 (+6.3%)          | 0.3029 (+1.8%)                | 0.3190 (+7.2%)                 |
| Precision@3   | 0.2118               | 0.2181 (+3.0%)          | 0.2261 (+6.8%)                | 0.2163 (+2.1%)                 |
| Precision@5   | 0.1828               | 0.1893 (+3.5%)          | 0.1984 (+8.5%)                | 0.1882 (+2.9%)                 |
| Precision@10  | 0.1338               | 0.1418 (+6.0%)          | 0.1649 (+23.2%)               | 0.1432 (+7.0%)                 |
| Precision@20  | 0.1005               | 0.1056 (+5.1%)          | 0.1282 (+27.5%)               | 0.1058 (+5.2%)                 |
| Precision@40  | 0.0690               | 0.0719 (+4.3%)          | 0.0864 (+25.3%)               | 0.0718 (+4.2%)                 |
| Precision@80  | 0.0426               | 0.0435 (+2.3%)          | 0.0498 (+17.0%)               | 0.0436 (+2.5%)                 |
| Precision@100 | 0.0358               | 0.0365 (+1.9%)          | 0.0410 (+14.3%)               | 0.0366 (+2.1%)                 |
| NDCG@1        | 0.2976               | 0.3164 (+6.3%)          | 0.3029 (+1.8%)                | 0.3190 (+7.2%)                 |
| NDCG@3        | 0.2508               | 0.2591 (+3.3%)          | 0.2620 (+4.5%)                | 0.2575 (+2.7%)                 |
| NDCG@5        | 0.2583               | 0.2677 (+3.7%)          | 0.2725 (+5.5%)                | 0.2661 (+3.0%)                 |
| NDCG@10       | 0.2868               | 0.3004 (+4.7%)          | 0.3252 (+13.4%)               | 0.3019 (+5.3%)                 |
| NDCG@20       | 0.3342               | 0.3497 (+4.7%)          | 0.3909 (+17.0%)               | 0.3495 (+4.6%)                 |
| NDCG@40       | 0.3777               | 0.3933 (+4.1%)          | 0.4393 (+16.3%)               | 0.3924 (+3.9%)                 |
| NDCG@80       | 0.4093               | 0.4232 (+3.4%)          | 0.4651 (+13.6%)               | 0.4230 (+3.3%)                 |
| NDCG@100      | 0.4174               | 0.4306 (+3.2%)          | 0.4701 (+12.6%)               | 0.4302 (+3.1%)                 |
| Hit Rate@1    | 0.2976               | 0.3164 (+6.3%)          | 0.3029 (+1.8%)                | 0.3190 (+7.2%)                 |
| Hit Rate@3    | 0.4638               | 0.4879 (+5.2%)          | 0.4987 (+7.5%)                | 0.4906 (+5.8%)                 |
| Hit Rate@5    | 0.5791               | 0.5791 (+0.0%)          | 0.5818 (+0.5%)                | 0.5791 (+0.0%)                 |
| Hit Rate@10   | 0.6810               | 0.7078 (+3.9%)          | 0.7239 (+6.3%)                | 0.7051 (+3.5%)                 |
| Hit Rate@20   | 0.7828               | 0.8257 (+5.5%)          | 0.8365 (+6.8%)                | 0.8231 (+5.1%)                 |
| Hit Rate@40   | 0.8633               | 0.8660 (+0.3%)          | 0.8686 (+0.6%)                | 0.8660 (+0.3%)                 |
| Hit Rate@80   | 0.8901               | 0.9008 (+1.2%)          | 0.9088 (+2.1%)                | 0.9008 (+1.2%)                 |
| Hit Rate@100  | 0.9008               | 0.9088 (+0.9%)          | 0.9169 (+1.8%)                | 0.9088 (+0.9%)                 |
| MRR           | 0.4214               | 0.4382 (+4.0%)          | 0.4349 (+3.2%)                | 0.4381 (+3.9%)                 |
| MAP           | 0.2412               | 0.2525 (+4.7%)          | 0.2848 (+18.1%)               | 0.2515 (+4.3%)                 |